# Carga y Preprocesamiento de Datos
* Carga los datos del archivo CSV utilizando NumPy.
* Realiza un preprocesamiento básico para asegurarte de que los datos estén limpios y listos para su análisis.



In [14]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')
ruta_csv = '/content/drive/My Drive/Colab Notebooks/Datos_tienda.csv'

Mounted at /content/drive


In [17]:
import numpy as np

def cargar_datos(ruta_archivo):
    datos = np.genfromtxt(ruta_csv, delimiter=',', skip_header=1, dtype=str)
    return datos

def total_ventas_por_producto_tienda(datos):
    productos = datos[:, 1]  # Columna de productos
    tiendas = datos[:, 3]  # Columna de tiendas
    ingresos = datos[:, 2].astype(float)  # Columna de ingresos

    # Obtener productos y tiendas únicos
    productos_unicos, idx_productos = np.unique(productos, return_inverse=True)
    tiendas_unicas, idx_tiendas = np.unique(tiendas, return_inverse=True)

    # Crear una matriz para almacenar las sumas de ingresos por producto y tienda
    total_ventas = np.zeros((len(productos_unicos), len(tiendas_unicas)))

    # Sumar los ingresos por producto y tienda
    for i in range(len(ingresos)):
        total_ventas[idx_productos[i], idx_tiendas[i]] += ingresos[i]

    return productos_unicos, tiendas_unicas, total_ventas

def promedio_ventas_por_producto_tienda(datos):
    productos = datos[:, 1]
    tiendas = datos[:, 3]
    fechas = datos[:, 0]
    ingresos = datos[:, 2].astype(float)

    # Obtener productos y tiendas únicos
    productos_unicos, idx_productos = np.unique(productos, return_inverse=True)
    tiendas_unicas, idx_tiendas = np.unique(tiendas, return_inverse=True)

    # Crear una matriz para almacenar los ingresos por día y tienda
    suma_ingresos = np.zeros((len(productos_unicos), len(tiendas_unicas)))
    conteo_dias = np.zeros((len(productos_unicos), len(tiendas_unicas)))

    # Usamos un array de tipo object para almacenar fechas
    fechas_unicas = np.empty_like(suma_ingresos, dtype=object)  # Cambiaamos a dtype=object porque no acepta string

    # Sumar ingresos y contar días únicos
    for i in range(len(ingresos)):
        prod_idx = idx_productos[i]
        tienda_idx = idx_tiendas[i]

        # Evitar contar la misma fecha más de una vez
        fecha_actual = fechas[i]
        if fechas_unicas[prod_idx, tienda_idx] != fecha_actual:
            conteo_dias[prod_idx, tienda_idx] += 1
            fechas_unicas[prod_idx, tienda_idx] = fecha_actual

        suma_ingresos[prod_idx, tienda_idx] += ingresos[i]

    # Calcular el promedio dividiendo el total de ventas entre los días
    promedio_ventas = np.divide(suma_ingresos, conteo_dias, out=np.zeros_like(suma_ingresos), where=conteo_dias!=0)

    return productos_unicos, tiendas_unicas, promedio_ventas

def identificar_mayores_menores_ventas(productos_unicos, tiendas_unicas, total_ventas):
    # Encontrar la venta máxima y mínima
    max_ventas = np.max(total_ventas)
    min_ventas = np.min(total_ventas)

    # Encontrar los índices de los valores máximos y mínimos
    max_indices = np.where(total_ventas == max_ventas)
    min_indices = np.where(total_ventas == min_ventas)

    # Obtener los productos y tiendas correspondientes
    productos_max = productos_unicos[max_indices[0]]
    tiendas_max = tiendas_unicas[max_indices[1]]

    productos_min = productos_unicos[min_indices[0]]
    tiendas_min = tiendas_unicas[min_indices[1]]

    return (productos_max, tiendas_max, max_ventas), (productos_min, tiendas_min, min_ventas)


if __name__ == "__main__":
    ruta_archivo = 'ruta/a/tu/csv.csv'
    datos = cargar_datos(ruta_archivo)

    # 1. Total de ventas por producto y tienda
    productos, tiendas, total_ventas = total_ventas_por_producto_tienda(datos)
    print("Total de ventas por producto y tienda:")
    print("\tFila = Producto\n\tColumna = Tienda")
    print(total_ventas)

    # 2. Promedio de ventas diarias por producto y tienda
    productos, tiendas, promedio_ventas = promedio_ventas_por_producto_tienda(datos)
    print("\nPromedio de ventas diarias por producto y tienda:")
    print("\tFila = Producto\n\tColumna = Tienda")
    print(promedio_ventas)

    # 3. Identificar mayores y menores ventas
    (productos_max, tiendas_max, max_ventas), (productos_min, tiendas_min, min_ventas) = identificar_mayores_menores_ventas(productos, tiendas, total_ventas)
    print(f"\nMayor venta: Producto(s) {productos_max} en Tienda(s) {tiendas_max} con {max_ventas}")
    print(f"Menor venta: Producto(s) {productos_min} en Tienda(s) {tiendas_min} con {min_ventas}")

Total de ventas por producto y tienda:
	Fila = Producto
	Columna = Tienda
[[184021. 257536. 182782. 179020. 252800.]
 [233455. 238245. 213360. 181220. 231919.]
 [205109. 234834. 297287. 285197. 237146.]
 [150657. 150886. 217435. 296018. 286935.]
 [181704. 205959. 227037. 238536. 292497.]
 [229855. 240966. 237230. 205372. 244626.]
 [177114. 222582. 231842. 280044. 259166.]
 [136485. 235626. 180537. 225678. 121361.]
 [243197. 212660. 266133. 173240. 242164.]
 [271102. 201378. 277122. 236367. 314294.]
 [306890. 223521. 241052. 195625. 244140.]
 [185301. 184614. 182351. 232699. 233052.]
 [224339. 258888. 203378. 329306. 236673.]
 [287781. 283259. 270294. 173381. 223229.]
 [289784. 270441. 243739. 207899. 245118.]
 [230033. 151903. 220926. 177332. 166971.]
 [210514. 278665. 246556. 270325. 225330.]
 [248832. 283479. 251026. 211609. 186402.]
 [326855. 228119. 167508. 196910. 269290.]
 [184070. 192929. 186097. 220502. 270382.]
 [174761. 267025. 266924. 249086. 179509.]
 [249589. 305478. 21437